In [1]:
import numpy as np

class Layer:
    def __init__(self, input_size=None, output_size=None, weights=None, bias=None):
        self.input = None
        self.output = None
        if (input_size is None or output_size is None) and (weights is None or bias is None):
            raise Exception('input and output sizes or weights with bias are required')
        if weights is None:
            self.weights = np.random.rand(input_size, output_size) # uniform
        else:
            self.weights = weights
            self.input_size = len(self.weights)
            self.output_size = len(self.weights[0])
        if bias is None:
            self.bias = np.random.rand(1, output_size)
        else:
            self.bias = bias

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        d_weights = -learning_rate * weights_error
        d_bias = -learning_rate * output_error
        return input_error, d_weights, d_bias
    
    def update_weights(self, d_weights, d_bias):
        self.weights += d_weights
        self.bias += d_bias
        
    
class ActivationLayer:
    def __init__(self, activation, activation_prime):
        self.input = None
        self.output = None
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [2]:
from copy import deepcopy

class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []

        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    def fit(self, x_train, y_train, epochs, learning_rate, batch_size):

        samples = len(x_train)
        
        batch_proportion = batch_size/samples

        for i in range(epochs):
            err = 0
            
            for batch_number in range(int(1/batch_proportion)):
                
                layersDeltaWeight = []
                layersDeltaBias = []
                
                for j in range(int(batch_number * batch_size), int((batch_number+1)*batch_size)):
                    
                    #forward propagation
                    
                    output = x_train[j]
                    
                    for layer in self.layers:
                        output = layer.forward_propagation(output)

                    err += self.loss(y_train[j], output)
                    
                    #backward propagation
                    
                    error = self.loss_prime(y_train[j], output)
                    
                    tmplayerDeltaWeights = []
                    tmplayerDeltaBias = []
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, ActivationLayer):
                            
                            error = layer.backward_propagation(error, learning_rate)
                            
                        else:
                            
                            error, deltaWeights, deltaBias = layer.backward_propagation(error, learning_rate)
                            tmplayerDeltaWeights.append(deltaWeights)
                            tmplayerDeltaBias.append(deltaBias)
                        
                    layersDeltaWeight.append(tmplayerDeltaWeights)
                    layersDeltaBias.append(tmplayerDeltaBias)
                    
                for layerdWeight, layerdBias in zip(layersDeltaWeight, layersDeltaBias):
                    
                    only_full_layers = []
                    
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, Layer):
                            
                            only_full_layers.append(layer)
                            
                    for dWeight, dBias, layer in zip(layerdWeight, layerdBias, only_full_layers):
                        
                        layer.update_weights(dWeight, dBias)

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))
            
            
    def fit_momentum(self, x_train, y_train, epochs, learning_rate, batch_size, lambd):

        samples = len(x_train)
        
        momentum_w = []
        momentum_b = []
        
        for layer in self.layers:
            
            if isinstance(layer, ActivationLayer):
                continue  
                
            momentum_w.append(deepcopy(layer.weights))
            momentum_b.append(deepcopy(layer.bias))
        
        momentum_w = np.array(momentum_w, dtype=object)
        
        for i in range(len(momentum_w)):
            momentum_w[i][:] = 0
            momentum_b[i][:] = 0

        batch_proportion = batch_size/samples

        for i in range(epochs):
            err = 0
            
            for batch_number in range(int(1/batch_proportion)):
                
                layersDeltaWeight = []
                layersDeltaBias = []
                
                for j in range(int(batch_number * batch_size), int((batch_number+1)*batch_size)):
                    
                    #forward propagation
                    
                    output = x_train[j]
                    
                    for layer in self.layers:
                        output = layer.forward_propagation(output)

                    err += self.loss(y_train[j], output)
                    
                    #backward propagation
                    
                    error = self.loss_prime(y_train[j], output)
                    
                    tmplayerDeltaWeights = []
                    tmplayerDeltaBias = []
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, ActivationLayer):
                            
                            error = layer.backward_propagation(error, learning_rate)
                            
                        else:
                            
                            error, deltaWeights, deltaBias = layer.backward_propagation(error, learning_rate)
                            tmplayerDeltaWeights.append(deltaWeights)
                            tmplayerDeltaBias.append(deltaBias)
                        
                    layersDeltaWeight.append(tmplayerDeltaWeights)
                    layersDeltaBias.append(tmplayerDeltaBias)
                    
                for layerdWeight, layerdBias in zip(layersDeltaWeight, layersDeltaBias):
                    
                    only_full_layers = []
                    
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, Layer):
                            
                            only_full_layers.append(layer)
                            
                    l = len(only_full_layers)
                            
                    for dWeight, dBias, layer in zip(layerdWeight, layerdBias, only_full_layers):
                        
                        l -= 1
                        momentum_w[l] = dWeight + learning_rate * momentum_w[l] * lambd
                        momentum_b[l] = dBias + learning_rate * momentum_b[l] * lambd
                        layer.update_weights(momentum_w[l], momentum_b[l])

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))
            
    def fit_RMSProp(self, x_train, y_train, epochs, learning_rate, batch_size, beta):
        
        samples = len(x_train)
        
        E_w = []
        E_b = []
        
        for layer in self.layers:
            
            if isinstance(layer, ActivationLayer):
                continue  
                
            E_w.append(deepcopy(layer.weights))
            E_b.append(deepcopy(layer.bias))
        
        E_w = np.array(E_w, dtype=object)
        
        for i in range(len(E_w)):
            E_w[i][:] = 0
            E_b[i][:] = 0
            
        batch_proportion = batch_size/samples

        for i in range(epochs):
            err = 0
            
            for batch_number in range(int(1/batch_proportion)):
                
                layersDeltaWeight = []
                layersDeltaBias = []
                
                for j in range(int(batch_number * batch_size), int((batch_number+1)*batch_size)):
                    
                    #forward propagation
                    
                    output = x_train[j]
                    
                    for layer in self.layers:
                        output = layer.forward_propagation(output)

                    err += self.loss(y_train[j], output)
                    
                    #backward propagation
                    
                    error = self.loss_prime(y_train[j], output)
                    
                    tmplayerDeltaWeights = []
                    tmplayerDeltaBias = []
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, ActivationLayer):
                            
                            error = layer.backward_propagation(error, learning_rate)
                            
                        else:
                            
                            error, deltaWeights, deltaBias = layer.backward_propagation(error, learning_rate)
                            tmplayerDeltaWeights.append(deltaWeights)
                            tmplayerDeltaBias.append(deltaBias)
                        
                    layersDeltaWeight.append(tmplayerDeltaWeights)
                    layersDeltaBias.append(tmplayerDeltaBias)
                    
                for layerdWeight, layerdBias in zip(layersDeltaWeight, layersDeltaBias):
                    
                    only_full_layers = []
                    
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, Layer):
                            
                            only_full_layers.append(layer)
                            
                    l = len(only_full_layers)
                            
                    for dWeight, dBias, layer in zip(layerdWeight, layerdBias, only_full_layers):
                        
                        l -= 1
                        E_w[l] = beta * E_w[l] + (1 - beta) * dWeight ** 2
                        E_b[l] = beta * E_b[l] + (1 - beta) * dBias ** 2
                        layer.update_weights(learning_rate * dWeight / np.sqrt(E_w[l]),
                                             learning_rate * dBias / np.sqrt(E_b[l]))

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [3]:
def sigmoid(x):
    e = np.exp((-1)*x)
    return 1/(1+e)

def derivative_sigmoid(x):
    e = np.exp((-1)*x)
    return e / ((1+e)**2)

def linear(x):
    return x

def derivative_linear(x):
    return np.array([1 for i in x])

In [4]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

# Square-large

In [5]:
import pandas as pd
data = pd.read_csv('mio1/regression/square-large-test.csv')
x_train = list(data.x)
y_train = list(data.y)
x_train = np.array([[[x]] for x in x_train])
y_train = np.array([[[y]] for y in y_train])

In [6]:
np.random.seed(2203)
net = Network()
net.add(Layer(1, 100))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(100, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit_momentum(x_train, y_train, epochs=10000, learning_rate=0.00002, batch_size=100, lambd=0.5)

epoch 1/10000   error=12459.760634
epoch 2/10000   error=11140.134158
epoch 3/10000   error=10669.008704
epoch 4/10000   error=10489.298170
epoch 5/10000   error=10417.055112
epoch 6/10000   error=10384.920157
epoch 7/10000   error=10367.502370
epoch 8/10000   error=10355.173169
epoch 9/10000   error=10344.262345
epoch 10/10000   error=10333.345380
epoch 11/10000   error=10321.841658
epoch 12/10000   error=10309.487900
epoch 13/10000   error=10296.141061
epoch 14/10000   error=10281.704658
epoch 15/10000   error=10266.101248
epoch 16/10000   error=10249.262195
epoch 17/10000   error=10231.123934
epoch 18/10000   error=10211.626667
epoch 19/10000   error=10190.713971
epoch 20/10000   error=10168.332683
epoch 21/10000   error=10144.432842
epoch 22/10000   error=10118.967560
epoch 23/10000   error=10091.892790
epoch 24/10000   error=10063.166957
epoch 25/10000   error=10032.750474
epoch 26/10000   error=10000.605156
epoch 27/10000   error=9966.693561
epoch 28/10000   error=9930.978303
epo

epoch 235/10000   error=294.861972
epoch 236/10000   error=292.546221
epoch 237/10000   error=290.260807
epoch 238/10000   error=288.004929
epoch 239/10000   error=285.777817
epoch 240/10000   error=283.578725
epoch 241/10000   error=281.406937
epoch 242/10000   error=279.261761
epoch 243/10000   error=277.142530
epoch 244/10000   error=275.048599
epoch 245/10000   error=272.979350
epoch 246/10000   error=270.934181
epoch 247/10000   error=268.912517
epoch 248/10000   error=266.913798
epoch 249/10000   error=264.937486
epoch 250/10000   error=262.983062
epoch 251/10000   error=261.050024
epoch 252/10000   error=259.137887
epoch 253/10000   error=257.246183
epoch 254/10000   error=255.374460
epoch 255/10000   error=253.522281
epoch 256/10000   error=251.689224
epoch 257/10000   error=249.874881
epoch 258/10000   error=248.078858
epoch 259/10000   error=246.300773
epoch 260/10000   error=244.540259
epoch 261/10000   error=242.796959
epoch 262/10000   error=241.070529
epoch 263/10000   er

epoch 476/10000   error=66.051127
epoch 477/10000   error=65.733881
epoch 478/10000   error=65.418952
epoch 479/10000   error=65.106319
epoch 480/10000   error=64.795965
epoch 481/10000   error=64.487868
epoch 482/10000   error=64.182011
epoch 483/10000   error=63.878375
epoch 484/10000   error=63.576941
epoch 485/10000   error=63.277690
epoch 486/10000   error=62.980604
epoch 487/10000   error=62.685665
epoch 488/10000   error=62.392854
epoch 489/10000   error=62.102154
epoch 490/10000   error=61.813548
epoch 491/10000   error=61.527016
epoch 492/10000   error=61.242542
epoch 493/10000   error=60.960109
epoch 494/10000   error=60.679699
epoch 495/10000   error=60.401296
epoch 496/10000   error=60.124882
epoch 497/10000   error=59.850441
epoch 498/10000   error=59.577956
epoch 499/10000   error=59.307411
epoch 500/10000   error=59.038789
epoch 501/10000   error=58.772074
epoch 502/10000   error=58.507251
epoch 503/10000   error=58.244303
epoch 504/10000   error=57.983214
epoch 505/1000

epoch 718/10000   error=27.836472
epoch 719/10000   error=27.765205
epoch 720/10000   error=27.694294
epoch 721/10000   error=27.623738
epoch 722/10000   error=27.553535
epoch 723/10000   error=27.483681
epoch 724/10000   error=27.414175
epoch 725/10000   error=27.345014
epoch 726/10000   error=27.276195
epoch 727/10000   error=27.207718
epoch 728/10000   error=27.139579
epoch 729/10000   error=27.071776
epoch 730/10000   error=27.004307
epoch 731/10000   error=26.937169
epoch 732/10000   error=26.870361
epoch 733/10000   error=26.803880
epoch 734/10000   error=26.737724
epoch 735/10000   error=26.671891
epoch 736/10000   error=26.606379
epoch 737/10000   error=26.541185
epoch 738/10000   error=26.476308
epoch 739/10000   error=26.411745
epoch 740/10000   error=26.347495
epoch 741/10000   error=26.283555
epoch 742/10000   error=26.219923
epoch 743/10000   error=26.156597
epoch 744/10000   error=26.093576
epoch 745/10000   error=26.030856
epoch 746/10000   error=25.968438
epoch 747/1000

epoch 960/10000   error=17.230292
epoch 961/10000   error=17.203657
epoch 962/10000   error=17.177107
epoch 963/10000   error=17.150642
epoch 964/10000   error=17.124261
epoch 965/10000   error=17.097964
epoch 966/10000   error=17.071750
epoch 967/10000   error=17.045619
epoch 968/10000   error=17.019570
epoch 969/10000   error=16.993604
epoch 970/10000   error=16.967718
epoch 971/10000   error=16.941914
epoch 972/10000   error=16.916191
epoch 973/10000   error=16.890548
epoch 974/10000   error=16.864985
epoch 975/10000   error=16.839501
epoch 976/10000   error=16.814097
epoch 977/10000   error=16.788771
epoch 978/10000   error=16.763523
epoch 979/10000   error=16.738354
epoch 980/10000   error=16.713262
epoch 981/10000   error=16.688246
epoch 982/10000   error=16.663308
epoch 983/10000   error=16.638446
epoch 984/10000   error=16.613659
epoch 985/10000   error=16.588948
epoch 986/10000   error=16.564313
epoch 987/10000   error=16.539752
epoch 988/10000   error=16.515265
epoch 989/1000

...

epoch 8973/10000   error=0.464890
epoch 8974/10000   error=0.464809
epoch 8975/10000   error=0.464727
epoch 8976/10000   error=0.464646
epoch 8977/10000   error=0.464564
epoch 8978/10000   error=0.464482
epoch 8979/10000   error=0.464401
epoch 8980/10000   error=0.464319
epoch 8981/10000   error=0.464238
epoch 8982/10000   error=0.464156
epoch 8983/10000   error=0.464075
epoch 8984/10000   error=0.463993
epoch 8985/10000   error=0.463912
epoch 8986/10000   error=0.463831
epoch 8987/10000   error=0.463749
epoch 8988/10000   error=0.463668
epoch 8989/10000   error=0.463587
epoch 8990/10000   error=0.463505
epoch 8991/10000   error=0.463424
epoch 8992/10000   error=0.463343
epoch 8993/10000   error=0.463262
epoch 8994/10000   error=0.463180
epoch 8995/10000   error=0.463099
epoch 8996/10000   error=0.463018
epoch 8997/10000   error=0.462937
epoch 8998/10000   error=0.462856
epoch 8999/10000   error=0.462775
epoch 9000/10000   error=0.462693
epoch 9001/10000   error=0.462612
epoch 9002/100

epoch 9215/10000   error=0.445758
epoch 9216/10000   error=0.445682
epoch 9217/10000   error=0.445605
epoch 9218/10000   error=0.445529
epoch 9219/10000   error=0.445452
epoch 9220/10000   error=0.445376
epoch 9221/10000   error=0.445300
epoch 9222/10000   error=0.445223
epoch 9223/10000   error=0.445147
epoch 9224/10000   error=0.445071
epoch 9225/10000   error=0.444994
epoch 9226/10000   error=0.444918
epoch 9227/10000   error=0.444842
epoch 9228/10000   error=0.444766
epoch 9229/10000   error=0.444689
epoch 9230/10000   error=0.444613
epoch 9231/10000   error=0.444537
epoch 9232/10000   error=0.444461
epoch 9233/10000   error=0.444385
epoch 9234/10000   error=0.444309
epoch 9235/10000   error=0.444233
epoch 9236/10000   error=0.444157
epoch 9237/10000   error=0.444080
epoch 9238/10000   error=0.444004
epoch 9239/10000   error=0.443928
epoch 9240/10000   error=0.443852
epoch 9241/10000   error=0.443777
epoch 9242/10000   error=0.443701
epoch 9243/10000   error=0.443625
epoch 9244/100

epoch 9458/10000   error=0.427780
epoch 9459/10000   error=0.427708
epoch 9460/10000   error=0.427637
epoch 9461/10000   error=0.427565
epoch 9462/10000   error=0.427494
epoch 9463/10000   error=0.427422
epoch 9464/10000   error=0.427351
epoch 9465/10000   error=0.427280
epoch 9466/10000   error=0.427208
epoch 9467/10000   error=0.427137
epoch 9468/10000   error=0.427065
epoch 9469/10000   error=0.426994
epoch 9470/10000   error=0.426923
epoch 9471/10000   error=0.426852
epoch 9472/10000   error=0.426780
epoch 9473/10000   error=0.426709
epoch 9474/10000   error=0.426638
epoch 9475/10000   error=0.426567
epoch 9476/10000   error=0.426495
epoch 9477/10000   error=0.426424
epoch 9478/10000   error=0.426353
epoch 9479/10000   error=0.426282
epoch 9480/10000   error=0.426211
epoch 9481/10000   error=0.426140
epoch 9482/10000   error=0.426069
epoch 9483/10000   error=0.425998
epoch 9484/10000   error=0.425927
epoch 9485/10000   error=0.425856
epoch 9486/10000   error=0.425785
epoch 9487/100

epoch 9702/10000   error=0.410908
epoch 9703/10000   error=0.410841
epoch 9704/10000   error=0.410774
epoch 9705/10000   error=0.410707
epoch 9706/10000   error=0.410641
epoch 9707/10000   error=0.410574
epoch 9708/10000   error=0.410507
epoch 9709/10000   error=0.410441
epoch 9710/10000   error=0.410374
epoch 9711/10000   error=0.410307
epoch 9712/10000   error=0.410241
epoch 9713/10000   error=0.410174
epoch 9714/10000   error=0.410107
epoch 9715/10000   error=0.410041
epoch 9716/10000   error=0.409974
epoch 9717/10000   error=0.409908
epoch 9718/10000   error=0.409841
epoch 9719/10000   error=0.409775
epoch 9720/10000   error=0.409708
epoch 9721/10000   error=0.409642
epoch 9722/10000   error=0.409575
epoch 9723/10000   error=0.409509
epoch 9724/10000   error=0.409442
epoch 9725/10000   error=0.409376
epoch 9726/10000   error=0.409309
epoch 9727/10000   error=0.409243
epoch 9728/10000   error=0.409177
epoch 9729/10000   error=0.409110
epoch 9730/10000   error=0.409044
epoch 9731/100

epoch 9944/10000   error=0.395272
epoch 9945/10000   error=0.395209
epoch 9946/10000   error=0.395147
epoch 9947/10000   error=0.395085
epoch 9948/10000   error=0.395022
epoch 9949/10000   error=0.394960
epoch 9950/10000   error=0.394897
epoch 9951/10000   error=0.394835
epoch 9952/10000   error=0.394773
epoch 9953/10000   error=0.394711
epoch 9954/10000   error=0.394648
epoch 9955/10000   error=0.394586
epoch 9956/10000   error=0.394524
epoch 9957/10000   error=0.394461
epoch 9958/10000   error=0.394399
epoch 9959/10000   error=0.394337
epoch 9960/10000   error=0.394275
epoch 9961/10000   error=0.394213
epoch 9962/10000   error=0.394151
epoch 9963/10000   error=0.394088
epoch 9964/10000   error=0.394026
epoch 9965/10000   error=0.393964
epoch 9966/10000   error=0.393902
epoch 9967/10000   error=0.393840
epoch 9968/10000   error=0.393778
epoch 9969/10000   error=0.393716
epoch 9970/10000   error=0.393654
epoch 9971/10000   error=0.393592
epoch 9972/10000   error=0.393530
epoch 9973/100

In [7]:
data = pd.read_csv('mio1/regression/square-large-training.csv')
x_test = list(data.x)
y_test = list(data.y)
x_test = np.array([[[x]] for x in x_test])
y_test = np.array([[[y]] for y in y_test])

In [8]:
out = net.predict(x_test)
mse(y_test, out)

0.37859503145479345

In [9]:
data = pd.read_csv('mio1/regression/square-large-test.csv')
x_train = list(data.x)
y_train = list(data.y)
x_train = np.array([[[x]] for x in x_train])
y_train = np.array([[[y]] for y in y_train])

In [10]:
np.random.seed(2203)
net = Network()
net.add(Layer(1, 100))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(100, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit_RMSProp(x_train, y_train, epochs=25000, learning_rate=0.00002, batch_size=100, beta=0.9)

epoch 1/25000   error=13195.074448
epoch 2/25000   error=13172.646245
epoch 3/25000   error=13150.858834
epoch 4/25000   error=13129.249368
epoch 5/25000   error=13107.815884
epoch 6/25000   error=13086.556709
epoch 7/25000   error=13065.470512
epoch 8/25000   error=13044.555964
epoch 9/25000   error=13023.811744
epoch 10/25000   error=13003.236537
epoch 11/25000   error=12982.829039
epoch 12/25000   error=12962.587952
epoch 13/25000   error=12942.511984
epoch 14/25000   error=12922.599854
epoch 15/25000   error=12902.850228
epoch 16/25000   error=12883.261370
epoch 17/25000   error=12863.831816
epoch 18/25000   error=12844.560334
epoch 19/25000   error=12825.445564
epoch 20/25000   error=12806.485576
epoch 21/25000   error=12787.678868
epoch 22/25000   error=12769.023788
epoch 23/25000   error=12750.519033
epoch 24/25000   error=12732.163445
epoch 25/25000   error=12713.955874
epoch 26/25000   error=12695.895175
epoch 27/25000   error=12677.980207
epoch 28/25000   error=12660.209755
e

epoch 226/25000   error=10867.774863
epoch 227/25000   error=10863.811454
epoch 228/25000   error=10859.876061
epoch 229/25000   error=10855.968492
epoch 230/25000   error=10852.088555
epoch 231/25000   error=10848.236063
epoch 232/25000   error=10844.410825
epoch 233/25000   error=10840.612654
epoch 234/25000   error=10836.841354
epoch 235/25000   error=10833.096675
epoch 236/25000   error=10829.378412
epoch 237/25000   error=10825.686385
epoch 238/25000   error=10822.020415
epoch 239/25000   error=10818.380324
epoch 240/25000   error=10814.765935
epoch 241/25000   error=10811.177060
epoch 242/25000   error=10807.613483
epoch 243/25000   error=10804.075023
epoch 244/25000   error=10800.561507
epoch 245/25000   error=10797.072766
epoch 246/25000   error=10793.608589
epoch 247/25000   error=10790.168766
epoch 248/25000   error=10786.753130
epoch 249/25000   error=10783.361516
epoch 250/25000   error=10779.993760
epoch 251/25000   error=10776.649700
epoch 252/25000   error=10773.329174
e

epoch 450/25000   error=10415.247887
epoch 451/25000   error=10414.389429
epoch 452/25000   error=10413.536514
epoch 453/25000   error=10412.689101
epoch 454/25000   error=10411.847148
epoch 455/25000   error=10411.010615
epoch 456/25000   error=10410.179461
epoch 457/25000   error=10409.353646
epoch 458/25000   error=10408.533129
epoch 459/25000   error=10407.717871
epoch 460/25000   error=10406.907832
epoch 461/25000   error=10406.102972
epoch 462/25000   error=10405.303253
epoch 463/25000   error=10404.508635
epoch 464/25000   error=10403.719079
epoch 465/25000   error=10402.934548
epoch 466/25000   error=10402.155003
epoch 467/25000   error=10401.380406
epoch 468/25000   error=10400.610718
epoch 469/25000   error=10399.845904
epoch 470/25000   error=10399.085925
epoch 471/25000   error=10398.330743
epoch 472/25000   error=10397.580324
epoch 473/25000   error=10396.834629
epoch 474/25000   error=10396.093623
epoch 475/25000   error=10395.357268
epoch 476/25000   error=10394.625530
e

epoch 675/25000   error=10304.777479
epoch 676/25000   error=10304.475340
epoch 677/25000   error=10304.173783
epoch 678/25000   error=10303.872798
epoch 679/25000   error=10303.572377
epoch 680/25000   error=10303.272510
epoch 681/25000   error=10302.973188
epoch 682/25000   error=10302.674402
epoch 683/25000   error=10302.376144
epoch 684/25000   error=10302.078403
epoch 685/25000   error=10301.781172
epoch 686/25000   error=10301.484442
epoch 687/25000   error=10301.188203
epoch 688/25000   error=10300.892447
epoch 689/25000   error=10300.597165
epoch 690/25000   error=10300.302349
epoch 691/25000   error=10300.007991
epoch 692/25000   error=10299.714081
epoch 693/25000   error=10299.420612
epoch 694/25000   error=10299.127575
epoch 695/25000   error=10298.834961
epoch 696/25000   error=10298.542763
epoch 697/25000   error=10298.250973
epoch 698/25000   error=10297.959581
epoch 699/25000   error=10297.668581
epoch 700/25000   error=10297.377964
epoch 701/25000   error=10297.087722
e

epoch 899/25000   error=10239.332514
epoch 900/25000   error=10239.011577
epoch 901/25000   error=10238.690152
epoch 902/25000   error=10238.368235
epoch 903/25000   error=10238.045825
epoch 904/25000   error=10237.722921
epoch 905/25000   error=10237.399519
epoch 906/25000   error=10237.075618
epoch 907/25000   error=10236.751217
epoch 908/25000   error=10236.426312
epoch 909/25000   error=10236.100903
epoch 910/25000   error=10235.774988
epoch 911/25000   error=10235.448565
epoch 912/25000   error=10235.121631
epoch 913/25000   error=10234.794185
epoch 914/25000   error=10234.466225
epoch 915/25000   error=10234.137750
epoch 916/25000   error=10233.808757
epoch 917/25000   error=10233.479246
epoch 918/25000   error=10233.149213
epoch 919/25000   error=10232.818657
epoch 920/25000   error=10232.487577
epoch 921/25000   error=10232.155970
epoch 922/25000   error=10231.823836
epoch 923/25000   error=10231.491172
epoch 924/25000   error=10231.157977
epoch 925/25000   error=10230.824248
e

...

epoch 23970/25000   error=40.492301
epoch 23971/25000   error=40.487744
epoch 23972/25000   error=40.483187
epoch 23973/25000   error=40.478630
epoch 23974/25000   error=40.474074
epoch 23975/25000   error=40.469517
epoch 23976/25000   error=40.464961
epoch 23977/25000   error=40.460404
epoch 23978/25000   error=40.455848
epoch 23979/25000   error=40.451292
epoch 23980/25000   error=40.446736
epoch 23981/25000   error=40.442180
epoch 23982/25000   error=40.437624
epoch 23983/25000   error=40.433068
epoch 23984/25000   error=40.428512
epoch 23985/25000   error=40.423956
epoch 23986/25000   error=40.419401
epoch 23987/25000   error=40.414845
epoch 23988/25000   error=40.410290
epoch 23989/25000   error=40.405735
epoch 23990/25000   error=40.401180
epoch 23991/25000   error=40.396625
epoch 23992/25000   error=40.392070
epoch 23993/25000   error=40.387515
epoch 23994/25000   error=40.382960
epoch 23995/25000   error=40.378405
epoch 23996/25000   error=40.373851
epoch 23997/25000   error=40

epoch 24199/25000   error=39.451475
epoch 24200/25000   error=39.446943
epoch 24201/25000   error=39.442412
epoch 24202/25000   error=39.437880
epoch 24203/25000   error=39.433349
epoch 24204/25000   error=39.428818
epoch 24205/25000   error=39.424287
epoch 24206/25000   error=39.419756
epoch 24207/25000   error=39.415225
epoch 24208/25000   error=39.410694
epoch 24209/25000   error=39.406164
epoch 24210/25000   error=39.401634
epoch 24211/25000   error=39.397103
epoch 24212/25000   error=39.392573
epoch 24213/25000   error=39.388043
epoch 24214/25000   error=39.383513
epoch 24215/25000   error=39.378984
epoch 24216/25000   error=39.374454
epoch 24217/25000   error=39.369925
epoch 24218/25000   error=39.365395
epoch 24219/25000   error=39.360866
epoch 24220/25000   error=39.356337
epoch 24221/25000   error=39.351808
epoch 24222/25000   error=39.347280
epoch 24223/25000   error=39.342751
epoch 24224/25000   error=39.338223
epoch 24225/25000   error=39.333694
epoch 24226/25000   error=39

epoch 24429/25000   error=38.413203
epoch 24430/25000   error=38.408709
epoch 24431/25000   error=38.404215
epoch 24432/25000   error=38.399722
epoch 24433/25000   error=38.395228
epoch 24434/25000   error=38.390735
epoch 24435/25000   error=38.386242
epoch 24436/25000   error=38.381750
epoch 24437/25000   error=38.377257
epoch 24438/25000   error=38.372765
epoch 24439/25000   error=38.368272
epoch 24440/25000   error=38.363781
epoch 24441/25000   error=38.359289
epoch 24442/25000   error=38.354797
epoch 24443/25000   error=38.350306
epoch 24444/25000   error=38.345815
epoch 24445/25000   error=38.341324
epoch 24446/25000   error=38.336833
epoch 24447/25000   error=38.332343
epoch 24448/25000   error=38.327852
epoch 24449/25000   error=38.323362
epoch 24450/25000   error=38.318872
epoch 24451/25000   error=38.314383
epoch 24452/25000   error=38.309893
epoch 24453/25000   error=38.305404
epoch 24454/25000   error=38.300915
epoch 24455/25000   error=38.296426
epoch 24456/25000   error=38

epoch 24657/25000   error=37.394437
epoch 24658/25000   error=37.389997
epoch 24659/25000   error=37.385558
epoch 24660/25000   error=37.381120
epoch 24661/25000   error=37.376681
epoch 24662/25000   error=37.372243
epoch 24663/25000   error=37.367805
epoch 24664/25000   error=37.363368
epoch 24665/25000   error=37.358931
epoch 24666/25000   error=37.354494
epoch 24667/25000   error=37.350057
epoch 24668/25000   error=37.345621
epoch 24669/25000   error=37.341184
epoch 24670/25000   error=37.336749
epoch 24671/25000   error=37.332313
epoch 24672/25000   error=37.327878
epoch 24673/25000   error=37.323443
epoch 24674/25000   error=37.319008
epoch 24675/25000   error=37.314574
epoch 24676/25000   error=37.310140
epoch 24677/25000   error=37.305706
epoch 24678/25000   error=37.301272
epoch 24679/25000   error=37.296839
epoch 24680/25000   error=37.292406
epoch 24681/25000   error=37.287973
epoch 24682/25000   error=37.283541
epoch 24683/25000   error=37.279109
epoch 24684/25000   error=37

epoch 24890/25000   error=36.368564
epoch 24891/25000   error=36.364202
epoch 24892/25000   error=36.359840
epoch 24893/25000   error=36.355479
epoch 24894/25000   error=36.351118
epoch 24895/25000   error=36.346758
epoch 24896/25000   error=36.342398
epoch 24897/25000   error=36.338038
epoch 24898/25000   error=36.333678
epoch 24899/25000   error=36.329320
epoch 24900/25000   error=36.324961
epoch 24901/25000   error=36.320603
epoch 24902/25000   error=36.316245
epoch 24903/25000   error=36.311888
epoch 24904/25000   error=36.307531
epoch 24905/25000   error=36.303174
epoch 24906/25000   error=36.298818
epoch 24907/25000   error=36.294462
epoch 24908/25000   error=36.290107
epoch 24909/25000   error=36.285752
epoch 24910/25000   error=36.281397
epoch 24911/25000   error=36.277043
epoch 24912/25000   error=36.272689
epoch 24913/25000   error=36.268336
epoch 24914/25000   error=36.263983
epoch 24915/25000   error=36.259631
epoch 24916/25000   error=36.255278
epoch 24917/25000   error=36

In [11]:
data = pd.read_csv('mio1/regression/square-large-training.csv')
x_test = list(data.x)
y_test = list(data.y)
x_test = np.array([[[x]] for x in x_test])
y_test = np.array([[[y]] for y in y_test])

out = net.predict(x_test)
mse(y_test, out)

34.52867800241231

# Steps-large

In [12]:
data = pd.read_csv('mio1/regression/steps-large-training.csv')
x_train = list(data.x)
y_train = list(data.y)
x_train = np.array([[[x]] for x in x_train])
y_train = np.array([[[y]] for y in y_train])

In [13]:
net = Network()
net.add(Layer(1, 100))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(100, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit_momentum(x_train, y_train, epochs=1250, learning_rate=0.00008, batch_size=100, lambd=0.5)

epoch 1/1250   error=1194.762949
epoch 2/1250   error=521.063776
epoch 3/1250   error=494.370187
epoch 4/1250   error=479.752901
epoch 5/1250   error=468.871671
epoch 6/1250   error=460.180974
epoch 7/1250   error=452.858948
epoch 8/1250   error=446.219431
epoch 9/1250   error=439.757794
epoch 10/1250   error=433.156722
epoch 11/1250   error=426.232084
epoch 12/1250   error=418.868675
epoch 13/1250   error=410.967655
epoch 14/1250   error=402.418061
epoch 15/1250   error=393.096192
epoch 16/1250   error=382.882698
epoch 17/1250   error=371.676821
epoch 18/1250   error=359.409957
epoch 19/1250   error=346.090142
epoch 20/1250   error=331.853112
epoch 21/1250   error=316.965407
epoch 22/1250   error=301.774334
epoch 23/1250   error=286.627761
epoch 24/1250   error=271.808241
epoch 25/1250   error=257.517322
epoch 26/1250   error=243.889979
epoch 27/1250   error=231.011852
epoch 28/1250   error=218.933151
epoch 29/1250   error=207.679128
epoch 30/1250   error=197.256866
epoch 31/1250   er

epoch 251/1250   error=52.323870
epoch 252/1250   error=52.231688
epoch 253/1250   error=52.140028
epoch 254/1250   error=52.048887
epoch 255/1250   error=51.958262
epoch 256/1250   error=51.868149
epoch 257/1250   error=51.778546
epoch 258/1250   error=51.689449
epoch 259/1250   error=51.600855
epoch 260/1250   error=51.512761
epoch 261/1250   error=51.425165
epoch 262/1250   error=51.338063
epoch 263/1250   error=51.251452
epoch 264/1250   error=51.165330
epoch 265/1250   error=51.079693
epoch 266/1250   error=50.994540
epoch 267/1250   error=50.909867
epoch 268/1250   error=50.825671
epoch 269/1250   error=50.741950
epoch 270/1250   error=50.658700
epoch 271/1250   error=50.575920
epoch 272/1250   error=50.493606
epoch 273/1250   error=50.411757
epoch 274/1250   error=50.330368
epoch 275/1250   error=50.249438
epoch 276/1250   error=50.168964
epoch 277/1250   error=50.088943
epoch 278/1250   error=50.009373
epoch 279/1250   error=49.930252
epoch 280/1250   error=49.851576
epoch 281/

epoch 500/1250   error=39.281273
epoch 501/1250   error=39.250079
epoch 502/1250   error=39.218967
epoch 503/1250   error=39.187939
epoch 504/1250   error=39.156992
epoch 505/1250   error=39.126127
epoch 506/1250   error=39.095344
epoch 507/1250   error=39.064641
epoch 508/1250   error=39.034019
epoch 509/1250   error=39.003476
epoch 510/1250   error=38.973014
epoch 511/1250   error=38.942631
epoch 512/1250   error=38.912327
epoch 513/1250   error=38.882101
epoch 514/1250   error=38.851954
epoch 515/1250   error=38.821885
epoch 516/1250   error=38.791893
epoch 517/1250   error=38.761978
epoch 518/1250   error=38.732140
epoch 519/1250   error=38.702379
epoch 520/1250   error=38.672694
epoch 521/1250   error=38.643084
epoch 522/1250   error=38.613550
epoch 523/1250   error=38.584091
epoch 524/1250   error=38.554707
epoch 525/1250   error=38.525397
epoch 526/1250   error=38.496161
epoch 527/1250   error=38.467000
epoch 528/1250   error=38.437911
epoch 529/1250   error=38.408896
epoch 530/

epoch 749/1250   error=33.400579
epoch 750/1250   error=33.382783
epoch 751/1250   error=33.365020
epoch 752/1250   error=33.347291
epoch 753/1250   error=33.329596
epoch 754/1250   error=33.311934
epoch 755/1250   error=33.294306
epoch 756/1250   error=33.276711
epoch 757/1250   error=33.259149
epoch 758/1250   error=33.241620
epoch 759/1250   error=33.224125
epoch 760/1250   error=33.206661
epoch 761/1250   error=33.189231
epoch 762/1250   error=33.171833
epoch 763/1250   error=33.154467
epoch 764/1250   error=33.137133
epoch 765/1250   error=33.119832
epoch 766/1250   error=33.102562
epoch 767/1250   error=33.085324
epoch 768/1250   error=33.068118
epoch 769/1250   error=33.050943
epoch 770/1250   error=33.033800
epoch 771/1250   error=33.016688
epoch 772/1250   error=32.999607
epoch 773/1250   error=32.982557
epoch 774/1250   error=32.965538
epoch 775/1250   error=32.948550
epoch 776/1250   error=32.931593
epoch 777/1250   error=32.914666
epoch 778/1250   error=32.897770
epoch 779/

epoch 998/1250   error=29.773572
epoch 999/1250   error=29.761553
epoch 1000/1250   error=29.749550
epoch 1001/1250   error=29.737564
epoch 1002/1250   error=29.725593
epoch 1003/1250   error=29.713638
epoch 1004/1250   error=29.701699
epoch 1005/1250   error=29.689775
epoch 1006/1250   error=29.677867
epoch 1007/1250   error=29.665975
epoch 1008/1250   error=29.654099
epoch 1009/1250   error=29.642237
epoch 1010/1250   error=29.630392
epoch 1011/1250   error=29.618562
epoch 1012/1250   error=29.606747
epoch 1013/1250   error=29.594948
epoch 1014/1250   error=29.583164
epoch 1015/1250   error=29.571395
epoch 1016/1250   error=29.559642
epoch 1017/1250   error=29.547903
epoch 1018/1250   error=29.536180
epoch 1019/1250   error=29.524472
epoch 1020/1250   error=29.512779
epoch 1021/1250   error=29.501101
epoch 1022/1250   error=29.489438
epoch 1023/1250   error=29.477790
epoch 1024/1250   error=29.466157
epoch 1025/1250   error=29.454539
epoch 1026/1250   error=29.442935
epoch 1027/1250 

epoch 1240/1250   error=27.266263
epoch 1241/1250   error=27.257318
epoch 1242/1250   error=27.248382
epoch 1243/1250   error=27.239456
epoch 1244/1250   error=27.230539
epoch 1245/1250   error=27.221632
epoch 1246/1250   error=27.212735
epoch 1247/1250   error=27.203847
epoch 1248/1250   error=27.194969
epoch 1249/1250   error=27.186100
epoch 1250/1250   error=27.177241


In [14]:
data = pd.read_csv('mio1/regression/steps-large-test.csv')
x_test = list(data.x)
y_test = list(data.y)
x_test = np.array([[[x]] for x in x_test])
y_test = np.array([[[y]] for y in y_test])
out = net.predict(x_test)
mse(y_test, out)

21.606021099233836

In [15]:
data = pd.read_csv('mio1/regression/steps-large-training.csv')
x_train = list(data.x)
y_train = list(data.y)
x_train = np.array([[[x]] for x in x_train])
y_train = np.array([[[y]] for y in y_train])

In [16]:
net = Network()
net.add(Layer(1, 100))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(100, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit_RMSProp(x_train, y_train, epochs=1500, learning_rate=0.00008, batch_size=10, beta=0.9)

epoch 1/1500   error=5653.921214
epoch 2/1500   error=4685.862512
epoch 3/1500   error=3845.960495
epoch 4/1500   error=3268.177369
epoch 5/1500   error=2917.613200
epoch 6/1500   error=2693.855246
epoch 7/1500   error=2535.576819
epoch 8/1500   error=2417.503210
epoch 9/1500   error=2328.119016
epoch 10/1500   error=2260.071017
epoch 11/1500   error=2207.730429
epoch 12/1500   error=2166.738301
epoch 13/1500   error=2133.834434
epoch 14/1500   error=2106.659193
epoch 15/1500   error=2083.540576
epoch 16/1500   error=2063.305762
epoch 17/1500   error=2045.133648
epoch 18/1500   error=2028.447424
epoch 19/1500   error=2012.839318
epoch 20/1500   error=1998.018808
epoch 21/1500   error=1983.777120
epoch 22/1500   error=1969.962809
epoch 23/1500   error=1956.464793
epoch 24/1500   error=1943.200415
epoch 25/1500   error=1930.106938
epoch 26/1500   error=1917.135373
epoch 27/1500   error=1904.245949
epoch 28/1500   error=1891.404718
epoch 29/1500   error=1878.581000
epoch 30/1500   error=1

epoch 242/1500   error=518.239996
epoch 243/1500   error=516.130580
epoch 244/1500   error=514.035735
epoch 245/1500   error=511.955369
epoch 246/1500   error=509.889395
epoch 247/1500   error=507.837735
epoch 248/1500   error=505.800315
epoch 249/1500   error=503.777068
epoch 250/1500   error=501.767930
epoch 251/1500   error=499.772847
epoch 252/1500   error=497.791767
epoch 253/1500   error=495.824642
epoch 254/1500   error=493.871429
epoch 255/1500   error=491.932089
epoch 256/1500   error=490.006587
epoch 257/1500   error=488.094888
epoch 258/1500   error=486.196962
epoch 259/1500   error=484.312780
epoch 260/1500   error=482.442314
epoch 261/1500   error=480.585538
epoch 262/1500   error=478.742424
epoch 263/1500   error=476.912947
epoch 264/1500   error=475.097080
epoch 265/1500   error=473.294796
epoch 266/1500   error=471.506067
epoch 267/1500   error=469.730862
epoch 268/1500   error=467.969150
epoch 269/1500   error=466.220897
epoch 270/1500   error=464.486069
epoch 271/1500

epoch 483/1500   error=288.997924
epoch 484/1500   error=288.614305
epoch 485/1500   error=288.231349
epoch 486/1500   error=287.849026
epoch 487/1500   error=287.467308
epoch 488/1500   error=287.086166
epoch 489/1500   error=286.705574
epoch 490/1500   error=286.325508
epoch 491/1500   error=285.945944
epoch 492/1500   error=285.566860
epoch 493/1500   error=285.188235
epoch 494/1500   error=284.810050
epoch 495/1500   error=284.432288
epoch 496/1500   error=284.054933
epoch 497/1500   error=283.677969
epoch 498/1500   error=283.301386
epoch 499/1500   error=282.925171
epoch 500/1500   error=282.549316
epoch 501/1500   error=282.173813
epoch 502/1500   error=281.798657
epoch 503/1500   error=281.423844
epoch 504/1500   error=281.049371
epoch 505/1500   error=280.675239
epoch 506/1500   error=280.301449
epoch 507/1500   error=279.928003
epoch 508/1500   error=279.554907
epoch 509/1500   error=279.182168
epoch 510/1500   error=278.809793
epoch 511/1500   error=278.437792
epoch 512/1500

epoch 724/1500   error=229.465903
epoch 725/1500   error=229.366068
epoch 726/1500   error=229.266500
epoch 727/1500   error=229.167143
epoch 728/1500   error=229.067937
epoch 729/1500   error=228.968811
epoch 730/1500   error=228.869685
epoch 731/1500   error=228.770467
epoch 732/1500   error=228.671047
epoch 733/1500   error=228.571301
epoch 734/1500   error=228.471081
epoch 735/1500   error=228.370212
epoch 736/1500   error=228.268492
epoch 737/1500   error=228.165682
epoch 738/1500   error=228.061513
epoch 739/1500   error=227.955680
epoch 740/1500   error=227.847858
epoch 741/1500   error=227.737733
epoch 742/1500   error=227.625044
epoch 743/1500   error=227.509673
epoch 744/1500   error=227.391760
epoch 745/1500   error=227.271823
epoch 746/1500   error=227.150834
epoch 747/1500   error=227.030149
epoch 748/1500   error=226.911235
epoch 749/1500   error=226.795251
epoch 750/1500   error=226.682711
epoch 751/1500   error=226.573444
epoch 752/1500   error=226.466867
epoch 753/1500

epoch 965/1500   error=205.633764
epoch 966/1500   error=205.523724
epoch 967/1500   error=205.413365
epoch 968/1500   error=205.302682
epoch 969/1500   error=205.191667
epoch 970/1500   error=205.080316
epoch 971/1500   error=204.968621
epoch 972/1500   error=204.856575
epoch 973/1500   error=204.744174
epoch 974/1500   error=204.631410
epoch 975/1500   error=204.518276
epoch 976/1500   error=204.404766
epoch 977/1500   error=204.290874
epoch 978/1500   error=204.176593
epoch 979/1500   error=204.061915
epoch 980/1500   error=203.946835
epoch 981/1500   error=203.831346
epoch 982/1500   error=203.715441
epoch 983/1500   error=203.599115
epoch 984/1500   error=203.482360
epoch 985/1500   error=203.365170
epoch 986/1500   error=203.247540
epoch 987/1500   error=203.129463
epoch 988/1500   error=203.010934
epoch 989/1500   error=202.891947
epoch 990/1500   error=202.772497
epoch 991/1500   error=202.652579
epoch 992/1500   error=202.532188
epoch 993/1500   error=202.411319
epoch 994/1500

epoch 1201/1500   error=173.452208
epoch 1202/1500   error=173.319299
epoch 1203/1500   error=173.186503
epoch 1204/1500   error=173.053824
epoch 1205/1500   error=172.921263
epoch 1206/1500   error=172.788825
epoch 1207/1500   error=172.656512
epoch 1208/1500   error=172.524326
epoch 1209/1500   error=172.392271
epoch 1210/1500   error=172.260348
epoch 1211/1500   error=172.128560
epoch 1212/1500   error=171.996910
epoch 1213/1500   error=171.865399
epoch 1214/1500   error=171.734029
epoch 1215/1500   error=171.602803
epoch 1216/1500   error=171.471721
epoch 1217/1500   error=171.340786
epoch 1218/1500   error=171.209998
epoch 1219/1500   error=171.079359
epoch 1220/1500   error=170.948870
epoch 1221/1500   error=170.818532
epoch 1222/1500   error=170.688343
epoch 1223/1500   error=170.558301
epoch 1224/1500   error=170.428405
epoch 1225/1500   error=170.298655
epoch 1226/1500   error=170.169052
epoch 1227/1500   error=170.039597
epoch 1228/1500   error=169.910290
epoch 1229/1500   er

epoch 1436/1500   error=149.281018
epoch 1437/1500   error=149.210130
epoch 1438/1500   error=149.139344
epoch 1439/1500   error=149.068657
epoch 1440/1500   error=148.998069
epoch 1441/1500   error=148.927579
epoch 1442/1500   error=148.857187
epoch 1443/1500   error=148.786896
epoch 1444/1500   error=148.716706
epoch 1445/1500   error=148.646621
epoch 1446/1500   error=148.576644
epoch 1447/1500   error=148.506780
epoch 1448/1500   error=148.437035
epoch 1449/1500   error=148.367416
epoch 1450/1500   error=148.297928
epoch 1451/1500   error=148.228582
epoch 1452/1500   error=148.159387
epoch 1453/1500   error=148.090351
epoch 1454/1500   error=148.021488
epoch 1455/1500   error=147.952807
epoch 1456/1500   error=147.884323
epoch 1457/1500   error=147.816047
epoch 1458/1500   error=147.747994
epoch 1459/1500   error=147.680177
epoch 1460/1500   error=147.612612
epoch 1461/1500   error=147.545312
epoch 1462/1500   error=147.478291
epoch 1463/1500   error=147.411566
epoch 1464/1500   er

In [17]:
data = pd.read_csv('mio1/regression/steps-large-test.csv')
x_test = list(data.x)
y_test = list(data.y)
x_test = np.array([[[x]] for x in x_test])
y_test = np.array([[[y]] for y in y_test])
out = net.predict(x_test)
mse(y_test, out)

148.88667854822936

# Multimodal-large

In [18]:
data = pd.read_csv('mio1/regression/multimodal-large-training.csv')
x_train = list(data.x)
y_train = list(data.y)
x_train = np.array([[[x]] for x in x_train])
y_train = np.array([[[y]] for y in y_train])

In [19]:
np.random.seed(2)

net = Network()
net.add(Layer(1, 100))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(100, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit_momentum(x_train, y_train, epochs=700, learning_rate=0.00005, batch_size=10, lambd=0.3)

epoch 1/700   error=4291.042927
epoch 2/700   error=3257.052501
epoch 3/700   error=2628.868620
epoch 4/700   error=2267.584172
epoch 5/700   error=2052.023200
epoch 6/700   error=1859.538126
epoch 7/700   error=1671.668320
epoch 8/700   error=1507.394197
epoch 9/700   error=1375.161414
epoch 10/700   error=1272.563552
epoch 11/700   error=1192.147720
epoch 12/700   error=1127.236911
epoch 13/700   error=1071.034936
epoch 14/700   error=1013.985488
epoch 15/700   error=947.743883
epoch 16/700   error=871.672972
epoch 17/700   error=793.425366
epoch 18/700   error=720.936623
epoch 19/700   error=657.118254
epoch 20/700   error=599.236466
epoch 21/700   error=549.315991
epoch 22/700   error=508.399760
epoch 23/700   error=474.941146
epoch 24/700   error=447.432187
epoch 25/700   error=424.537163
epoch 26/700   error=405.174569
epoch 27/700   error=388.508443
epoch 28/700   error=373.901807
epoch 29/700   error=360.874429
epoch 30/700   error=349.070268
epoch 31/700   error=338.229283
epo

epoch 258/700   error=15.292769
epoch 259/700   error=15.217575
epoch 260/700   error=15.143386
epoch 261/700   error=15.070181
epoch 262/700   error=14.997940
epoch 263/700   error=14.926643
epoch 264/700   error=14.856270
epoch 265/700   error=14.786804
epoch 266/700   error=14.718224
epoch 267/700   error=14.650513
epoch 268/700   error=14.583654
epoch 269/700   error=14.517628
epoch 270/700   error=14.452420
epoch 271/700   error=14.388012
epoch 272/700   error=14.324388
epoch 273/700   error=14.261532
epoch 274/700   error=14.199430
epoch 275/700   error=14.138065
epoch 276/700   error=14.077423
epoch 277/700   error=14.017489
epoch 278/700   error=13.958250
epoch 279/700   error=13.899691
epoch 280/700   error=13.841799
epoch 281/700   error=13.784561
epoch 282/700   error=13.727963
epoch 283/700   error=13.671994
epoch 284/700   error=13.616640
epoch 285/700   error=13.561889
epoch 286/700   error=13.507730
epoch 287/700   error=13.454151
epoch 288/700   error=13.401141
epoch 28

epoch 519/700   error=7.566392
epoch 520/700   error=7.555780
epoch 521/700   error=7.545247
epoch 522/700   error=7.534793
epoch 523/700   error=7.524418
epoch 524/700   error=7.514119
epoch 525/700   error=7.503897
epoch 526/700   error=7.493750
epoch 527/700   error=7.483677
epoch 528/700   error=7.473679
epoch 529/700   error=7.463753
epoch 530/700   error=7.453900
epoch 531/700   error=7.444118
epoch 532/700   error=7.434408
epoch 533/700   error=7.424768
epoch 534/700   error=7.415198
epoch 535/700   error=7.405697
epoch 536/700   error=7.396266
epoch 537/700   error=7.386902
epoch 538/700   error=7.377607
epoch 539/700   error=7.368378
epoch 540/700   error=7.359217
epoch 541/700   error=7.350121
epoch 542/700   error=7.341092
epoch 543/700   error=7.332128
epoch 544/700   error=7.323229
epoch 545/700   error=7.314395
epoch 546/700   error=7.305624
epoch 547/700   error=7.296918
epoch 548/700   error=7.288275
epoch 549/700   error=7.279694
epoch 550/700   error=7.271176
epoch 55

In [20]:
data = pd.read_csv('mio1/regression/multimodal-large-test.csv')
x_test = list(data.x)
y_test = list(data.y)
x_test = np.array([[[x]] for x in x_test])
y_test = np.array([[[y]] for y in y_test])
out = net.predict(x_test)
mse(y_test, out)

2.371728969090831

In [21]:
data = pd.read_csv('mio1/regression/multimodal-large-training.csv')
x_train = list(data.x)
y_train = list(data.y)
x_train = np.array([[[x]] for x in x_train])
y_train = np.array([[[y]] for y in y_train])

In [26]:
np.random.seed(2)

net = Network()
net.add(Layer(1, 150))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(150, 1))
net.add(ActivationLayer(linear, derivative_linear))

net.use(mse, mse_prime)
net.fit_RMSProp(x_train, y_train, epochs=6000, learning_rate=0.00008, batch_size=100, beta=0.9)

epoch 1/6000   error=5359.901712
epoch 2/6000   error=4943.808512
epoch 3/6000   error=4806.070282
epoch 4/6000   error=4694.535567
epoch 5/6000   error=4582.540848
epoch 6/6000   error=4470.124478
epoch 7/6000   error=4361.326153
epoch 8/6000   error=4260.016831
epoch 9/6000   error=4168.626440
epoch 10/6000   error=4087.994098
epoch 11/6000   error=4017.764580
epoch 12/6000   error=3956.905626
epoch 13/6000   error=3904.122886
epoch 14/6000   error=3858.118914
epoch 15/6000   error=3817.723629
epoch 16/6000   error=3781.941981
epoch 17/6000   error=3749.956627
epoch 18/6000   error=3721.109982
epoch 19/6000   error=3694.879227
epoch 20/6000   error=3670.850840
epoch 21/6000   error=3648.697327
epoch 22/6000   error=3628.157028
epoch 23/6000   error=3609.017260
epoch 24/6000   error=3591.100776
epoch 25/6000   error=3574.255305
epoch 26/6000   error=3558.345786
epoch 27/6000   error=3543.248957
epoch 28/6000   error=3528.850013
epoch 29/6000   error=3515.041015
epoch 30/6000   error=3

epoch 239/6000   error=1399.146455
epoch 240/6000   error=1396.458460
epoch 241/6000   error=1393.771121
epoch 242/6000   error=1391.083838
epoch 243/6000   error=1388.395937
epoch 244/6000   error=1385.706681
epoch 245/6000   error=1383.015268
epoch 246/6000   error=1380.320840
epoch 247/6000   error=1377.622489
epoch 248/6000   error=1374.919258
epoch 249/6000   error=1372.210150
epoch 250/6000   error=1369.494124
epoch 251/6000   error=1366.770103
epoch 252/6000   error=1364.036970
epoch 253/6000   error=1361.293564
epoch 254/6000   error=1358.538686
epoch 255/6000   error=1355.771098
epoch 256/6000   error=1352.989525
epoch 257/6000   error=1350.192654
epoch 258/6000   error=1347.379122
epoch 259/6000   error=1344.547522
epoch 260/6000   error=1341.696396
epoch 261/6000   error=1338.824250
epoch 262/6000   error=1335.929559
epoch 263/6000   error=1333.010784
epoch 264/6000   error=1330.066375
epoch 265/6000   error=1327.094790
epoch 266/6000   error=1324.094496
epoch 267/6000   err

epoch 478/6000   error=571.855877
epoch 479/6000   error=570.911090
epoch 480/6000   error=569.975727
epoch 481/6000   error=569.049372
epoch 482/6000   error=568.131631
epoch 483/6000   error=567.222135
epoch 484/6000   error=566.320561
epoch 485/6000   error=565.426688
epoch 486/6000   error=564.540410
epoch 487/6000   error=563.661380
epoch 488/6000   error=562.789177
epoch 489/6000   error=561.923438
epoch 490/6000   error=561.063833
epoch 491/6000   error=560.210057
epoch 492/6000   error=559.361830
epoch 493/6000   error=558.518895
epoch 494/6000   error=557.681016
epoch 495/6000   error=556.847982
epoch 496/6000   error=556.019609
epoch 497/6000   error=555.195728
epoch 498/6000   error=554.376182
epoch 499/6000   error=553.560824
epoch 500/6000   error=552.749517
epoch 501/6000   error=551.942132
epoch 502/6000   error=551.138540
epoch 503/6000   error=550.338590
epoch 504/6000   error=549.542100
epoch 505/6000   error=548.748871
epoch 506/6000   error=547.958690
epoch 507/6000

epoch 719/6000   error=432.352516
epoch 720/6000   error=431.976540
epoch 721/6000   error=431.602113
epoch 722/6000   error=431.229237
epoch 723/6000   error=430.857942
epoch 724/6000   error=430.488323
epoch 725/6000   error=430.120468
epoch 726/6000   error=429.754388
epoch 727/6000   error=429.390068
epoch 728/6000   error=429.027493
epoch 729/6000   error=428.666654
epoch 730/6000   error=428.307549
epoch 731/6000   error=427.950177
epoch 732/6000   error=427.594541
epoch 733/6000   error=427.240643
epoch 734/6000   error=426.888477
epoch 735/6000   error=426.538027
epoch 736/6000   error=426.189276
epoch 737/6000   error=425.842205
epoch 738/6000   error=425.496792
epoch 739/6000   error=425.153016
epoch 740/6000   error=424.810861
epoch 741/6000   error=424.470336
epoch 742/6000   error=424.131422
epoch 743/6000   error=423.794017
epoch 744/6000   error=423.458017
epoch 745/6000   error=423.123408
epoch 746/6000   error=422.790220
epoch 747/6000   error=422.458425
epoch 748/6000

epoch 960/6000   error=367.094231
epoch 961/6000   error=366.889790
epoch 962/6000   error=366.685752
epoch 963/6000   error=366.482113
epoch 964/6000   error=366.278868
epoch 965/6000   error=366.076011
epoch 966/6000   error=365.873538
epoch 967/6000   error=365.671442
epoch 968/6000   error=365.469720
epoch 969/6000   error=365.268369
epoch 970/6000   error=365.067388
epoch 971/6000   error=364.866773
epoch 972/6000   error=364.666519
epoch 973/6000   error=364.466620
epoch 974/6000   error=364.267066
epoch 975/6000   error=364.067849
epoch 976/6000   error=363.868960
epoch 977/6000   error=363.670392
epoch 978/6000   error=363.472137
epoch 979/6000   error=363.274190
epoch 980/6000   error=363.076544
epoch 981/6000   error=362.879194
epoch 982/6000   error=362.682133
epoch 983/6000   error=362.485357
epoch 984/6000   error=362.288858
epoch 985/6000   error=362.092631
epoch 986/6000   error=361.896670
epoch 987/6000   error=361.700969
epoch 988/6000   error=361.505522
epoch 989/6000

...

epoch 4956/6000   error=287.736666
epoch 4957/6000   error=287.709954
epoch 4958/6000   error=287.683231
epoch 4959/6000   error=287.656493
epoch 4960/6000   error=287.629733
epoch 4961/6000   error=287.602953
epoch 4962/6000   error=287.576161
epoch 4963/6000   error=287.549360
epoch 4964/6000   error=287.522547
epoch 4965/6000   error=287.495717
epoch 4966/6000   error=287.468873
epoch 4967/6000   error=287.442019
epoch 4968/6000   error=287.415153
epoch 4969/6000   error=287.388277
epoch 4970/6000   error=287.361390
epoch 4971/6000   error=287.334491
epoch 4972/6000   error=287.307582
epoch 4973/6000   error=287.280662
epoch 4974/6000   error=287.253730
epoch 4975/6000   error=287.226787
epoch 4976/6000   error=287.199832
epoch 4977/6000   error=287.172865
epoch 4978/6000   error=287.145886
epoch 4979/6000   error=287.118894
epoch 4980/6000   error=287.091891
epoch 4981/6000   error=287.064874
epoch 4982/6000   error=287.037844
epoch 4983/6000   error=287.010802
epoch 4984/6000   er

epoch 5191/6000   error=281.990249
epoch 5192/6000   error=281.968843
epoch 5193/6000   error=281.947457
epoch 5194/6000   error=281.926090
epoch 5195/6000   error=281.904741
epoch 5196/6000   error=281.883409
epoch 5197/6000   error=281.862093
epoch 5198/6000   error=281.840794
epoch 5199/6000   error=281.819510
epoch 5200/6000   error=281.798241
epoch 5201/6000   error=281.776988
epoch 5202/6000   error=281.755749
epoch 5203/6000   error=281.734524
epoch 5204/6000   error=281.713314
epoch 5205/6000   error=281.692119
epoch 5206/6000   error=281.670938
epoch 5207/6000   error=281.649772
epoch 5208/6000   error=281.628621
epoch 5209/6000   error=281.607485
epoch 5210/6000   error=281.586364
epoch 5211/6000   error=281.565258
epoch 5212/6000   error=281.544168
epoch 5213/6000   error=281.523094
epoch 5214/6000   error=281.502036
epoch 5215/6000   error=281.480995
epoch 5216/6000   error=281.459975
epoch 5217/6000   error=281.438943
epoch 5218/6000   error=281.417960
epoch 5219/6000   er

epoch 5426/6000   error=277.849828
epoch 5427/6000   error=277.835531
epoch 5428/6000   error=277.821216
epoch 5429/6000   error=277.806885
epoch 5430/6000   error=277.792528
epoch 5431/6000   error=277.778145
epoch 5432/6000   error=277.763734
epoch 5433/6000   error=277.749292
epoch 5434/6000   error=277.734818
epoch 5435/6000   error=277.720308
epoch 5436/6000   error=277.705761
epoch 5437/6000   error=277.691173
epoch 5438/6000   error=277.676544
epoch 5439/6000   error=277.661869
epoch 5440/6000   error=277.647148
epoch 5441/6000   error=277.632377
epoch 5442/6000   error=277.617555
epoch 5443/6000   error=277.602680
epoch 5444/6000   error=277.587749
epoch 5445/6000   error=277.572761
epoch 5446/6000   error=277.557713
epoch 5447/6000   error=277.542603
epoch 5448/6000   error=277.527430
epoch 5449/6000   error=277.512191
epoch 5450/6000   error=277.496884
epoch 5451/6000   error=277.481508
epoch 5452/6000   error=277.466061
epoch 5453/6000   error=277.450541
epoch 5454/6000   er

epoch 5661/6000   error=271.645205
epoch 5662/6000   error=271.610515
epoch 5663/6000   error=271.575879
epoch 5664/6000   error=271.541296
epoch 5665/6000   error=271.506767
epoch 5666/6000   error=271.472291
epoch 5667/6000   error=271.437867
epoch 5668/6000   error=271.403496
epoch 5669/6000   error=271.369177
epoch 5670/6000   error=271.334908
epoch 5671/6000   error=271.300690
epoch 5672/6000   error=271.266522
epoch 5673/6000   error=271.232404
epoch 5674/6000   error=271.198334
epoch 5675/6000   error=271.164312
epoch 5676/6000   error=271.130338
epoch 5677/6000   error=271.096410
epoch 5678/6000   error=271.062527
epoch 5679/6000   error=271.028690
epoch 5680/6000   error=270.994897
epoch 5681/6000   error=270.961147
epoch 5682/6000   error=270.927439
epoch 5683/6000   error=270.893772
epoch 5684/6000   error=270.860145
epoch 5685/6000   error=270.826556
epoch 5686/6000   error=270.793005
epoch 5687/6000   error=270.759489
epoch 5688/6000   error=270.726009
epoch 5689/6000   er

epoch 5896/6000   error=264.934666
epoch 5897/6000   error=264.914868
epoch 5898/6000   error=264.895145
epoch 5899/6000   error=264.875504
epoch 5900/6000   error=264.855952
epoch 5901/6000   error=264.836498
epoch 5902/6000   error=264.817150
epoch 5903/6000   error=264.797871
epoch 5904/6000   error=264.778722
epoch 5905/6000   error=264.759737
epoch 5906/6000   error=264.740926
epoch 5907/6000   error=264.722295
epoch 5908/6000   error=264.703845
epoch 5909/6000   error=264.685578
epoch 5910/6000   error=264.667493
epoch 5911/6000   error=264.649591
epoch 5912/6000   error=264.631869
epoch 5913/6000   error=264.614329
epoch 5914/6000   error=264.596970
epoch 5915/6000   error=264.579792
epoch 5916/6000   error=264.562796
epoch 5917/6000   error=264.545982
epoch 5918/6000   error=264.529351
epoch 5919/6000   error=264.512903
epoch 5920/6000   error=264.496638
epoch 5921/6000   error=264.480556
epoch 5922/6000   error=264.464659
epoch 5923/6000   error=264.448945
epoch 5924/6000   er

In [27]:
data = pd.read_csv('mio1/regression/multimodal-large-test.csv')
x_test = list(data.x)
y_test = list(data.y)
x_test = np.array([[[x]] for x in x_test])
y_test = np.array([[[y]] for y in y_test])
out = net.predict(x_test)
mse(y_test, out)

279.63794030834606